In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import cv2
from scipy.ndimage import label
from skimage.io import imsave, imread
import tqdm
import warnings
import os

import zipfile

In [ ]:
!ls ../input

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
z = zipfile.ZipFile('../input/0417-external-rare-cell-mask-generation/external_mask.zip')

In [ ]:
def squarify(M,val):
    (a,b,c)=M.shape
    if a>b:
        padding=((0,0),((a-b)//2,a-b-(a-b)//2),(0, 0))
    else:
        padding=(((b-a)//2,b-a-(b-a)//2),(0,0),(0, 0))
    return np.pad(M,padding,mode='constant',constant_values=val)

In [ ]:
compress = zipfile.ZipFile('./result.zip', 'w')

In [ ]:
SZ = 2048


for x in tqdm.tqdm_notebook(z.namelist()[:10000]):
    if 'npz' in x:
        iid = x.split('/')[1].split('.')[0]
        mask = np.load(z.open(x))['arr_0']

        r = cv2.imread(f'../input/hpa-0417-rare-jpeg-collect/publichpa/{iid}_red.jpg', 0)
        g = cv2.imread(f'../input/hpa-0417-rare-jpeg-collect/publichpa/{iid}_green.jpg', 0)
        b = cv2.imread(f'../input/hpa-0417-rare-jpeg-collect/publichpa/{iid}_blue.jpg', 0)
        a = cv2.imread(f'../input/hpa-0417-rare-jpeg-collect/publichpa/{iid}_yellow.jpg', 0)
        if not r.shape[0] == SZ:
            r = cv2.resize(r, (SZ, SZ))
            g = cv2.resize(g, (SZ, SZ))
            b = cv2.resize(b, (SZ, SZ))
            a = cv2.resize(a, (SZ, SZ))
        img = np.stack([r, g, b, a], -1)

        for i in range(1, mask.max()):
            sub_mask = cv2.resize((mask == i).astype(np.uint8), (SZ, SZ))
            x, y = np.where(sub_mask == 1)
            sub = img[x.min(): x.max(), y.min(): y.max()]
            crop_sub_mask = sub_mask[x.min(): x.max(), y.min(): y.max()]
            crop_sub_mask = np.repeat(crop_sub_mask[:, :, np.newaxis], 4, axis=2)
            r = sub * crop_sub_mask
            r = squarify(r, 0)
            if r.shape[0] > 256:
                r = cv2.resize(r, (256, 256))
            imsave(f'./{iid}_{i}.png', r)
            compress.write(f'./{iid}_{i}.png')
            os.remove(f'./{iid}_{i}.png')